## Uppgift 2 - Sport statistics

Date: 2021-11-10

Choose 2-4 sports and create suitable graphs / diagrams to visualize for example:
- medal distribution between countries in sports
- age distribution in sports
- Create more plots to visualize different aspects of the sport.

In [1]:
import pandas as pd

import dash
from dash import dcc, html
from dash.dependencies import Output, Input
import dash_bootstrap_components as dbc

import plotly_express as px
import seaborn as sns
from matplotlib import pyplot as plt

import analyze_functions as af

In [2]:
# Import data
athlete_regions = pd.read_csv("data/athlete_regions.csv")
athlete_regions.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes,_merge
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,both
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,both
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,both
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,both
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,both


In [3]:
# Import data
athlete_iso = pd.read_csv("data/athlete_iso.csv").iloc[:, 1:]
athlete_iso.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,Country,ISO
0,1076,Jammal-ud-Din Affendi,M,28.0,NaN,NaN,Afghanistan,AFG,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,Afghanistan,Afghanistan,AFG
1,133717,Ali Ahmed Youssef Zaid,M,20.0,168.0,58.0,Afghanistan,AFG,1960 Summer,1960,Summer,Roma,Athletics,Athletics Men's 200 metres,NaN,Afghanistan,Afghanistan,AFG
2,502,Ahmad Shah Abouwi,M,NaN,NaN,NaN,Afghanistan,AFG,1956 Summer,1956,Summer,Melbourne,Hockey,Hockey Men's Hockey,NaN,Afghanistan,Afghanistan,AFG
3,5841,Sayed Ali Atta,M,22.0,NaN,NaN,Afghanistan,AFG,1936 Summer,1936,Summer,Berlin,Hockey,Hockey Men's Hockey,NaN,Afghanistan,Afghanistan,AFG
4,1745,Mohammad Aktar,M,17.0,156.0,48.0,Afghanistan,AFG,1980 Summer,1980,Summer,Moskva,Wrestling,"Wrestling Men's Light-Flyweight, Freestyle",NaN,Afghanistan,Afghanistan,AFG


In [4]:
# get noc_iso data
noc_iso = pd.read_csv("data/noc_iso.csv").iloc[:, 1:]
noc_iso.head()

,Country,NOC,ISO
0,Afghanistan,AFG,AFG
1,Albania,ALB,ALB
2,Algeria,ALG,DZA
3,American Samoa,ASA,ASM
4,Andorra,AND,AND


**1. medal distribution between countries (all sports)**

1.1 plot sum of medals in the history on map

In [5]:
df_all = af.count_medals_n(athlete_iso, "Country", "ISO")
df_all.head()

Medal,Country,ISO,Bronze,Gold,Silver,Total
0,Afghanistan,AFG,2,0,0,2
1,Algeria,DZA,8,5,4,17
2,Argentina,ARG,91,91,92,274
3,Armenia,ARM,9,2,5,16
4,Australia,AUS,517,348,455,1320


In [6]:
for medal in ["Total", "Gold", "Silver","Bronze"]:

    fig = px.choropleth(df_all, locations="ISO",
                        color=medal,
                        scope=None,
                        hover_name="Country",
                        title = f"Geographic map on sum of {medal} medals", 
                        range_color=[0,df_all[medal].quantile(0.95)],
                        color_continuous_scale=px.colors.sequential.Plasma)
    
    
    fig["layout"].pop("updatemenus")
    fig.show()
    fig.write_html(f'Visualiseringar/Q2.plot_sum_of_{medal}_on_map.html', auto_open=True)

1.2 Bar plot of top ten countries

In [7]:
# for all sports, chosen medals
medal ="Total"
temp = df_all.sort_values(medal, ascending=False)
top10_all = temp.head(10)
top10_all.head()

Medal,Country,ISO,Bronze,Gold,Silver,Total
128,United States,USA,1358,2638,1641,5637
100,Russia,RUS,1178,1599,1170,3947
41,Germany,DEU,746,745,674,2165
127,United Kingdom,GBR,651,678,739,2068
38,France,FRA,666,501,610,1777


In [8]:
fig = px.bar(top10_all, y="Country", x=["Total", "Gold", "Silver", "Bronze"],
             title="top 10 countries by sum of medals")
fig.update_layout(barmode='group')
fig.show()

In [9]:
# for all sports, chosen medals
medal ="Gold"
temp = df_all.sort_values(medal, ascending=False)
top10_all = temp.head(10)
top10_all.head()

Medal,Country,ISO,Bronze,Gold,Silver,Total
128,United States,USA,1358,2638,1641,5637
100,Russia,RUS,1178,1599,1170,3947
41,Germany,DEU,746,745,674,2165
127,United Kingdom,GBR,651,678,739,2068
57,Italy,ITA,531,575,531,1637


In [10]:
fig = px.bar(top10_all, y="Country", x="Gold",
             title="top 10 countries by sum of gold medals")
fig.show()


**2. medal distribution between countries (specific sport)**

The sports that we are going to check are: ice hockey, tennis, swimming, and fotboll

2.1 plot on map

In [11]:
athlete_iso = pd.read_csv("data/athlete_iso.csv").iloc[:, 1:]
df_sports = af.count_medals_n(athlete_iso, "Country", "ISO", "Sport")
df_sports.head()

Medal,Country,ISO,Sport,Bronze,Gold,Silver,Total
0,Afghanistan,AFG,Taekwondo,2,0,0,2
1,Algeria,DZA,Athletics,2,4,3,9
2,Algeria,DZA,Boxing,5,1,0,6
3,Algeria,DZA,Judo,1,0,1,2
4,Argentina,ARG,Athletics,0,2,3,5


In [12]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
medal = "Total"
for sport in sports:
    df_sport = df_sports[df_sports["Sport"] == sport]
    fig = px.choropleth(df_sport, locations="ISO",
                        color=medal,
                        scope=None,
                        hover_name="Country",
                        title = f"Geographic map on sum of  {sport} {medal} medals", 
                        range_color=[0,df_sport[medal].quantile(0.95)],
                        color_continuous_scale=px.colors.sequential.Plasma)
    
    
    fig["layout"].pop("updatemenus")
    fig.show()
    fig.write_html(f'Visualiseringar/Q2.plot_sum_of_{sport}_{medal}_on_map.html', auto_open=True)

2.2 bar pot of top 10 countries in specific sport

In [13]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
medal = "Total"
for sport in sports:
    df_sport = df_sports[df_sports["Sport"] == sport]
    df_temp = df_sport.sort_values(medal, ascending=False)
    top10_sport = df_temp.head(10)
    
    # plot:

    fig = px.bar(top10_sport, y="Country", x=["Total", "Gold", "Silver", "Bronze"],
             title=f"Top 10 countries of medals in {sport} medals")
    fig.update_layout(barmode='group')
    fig.show()
    #fig.write_html(f"Visualiseringar/Q2.{sport}_country.html", auto_open=True)

**3.Medal distribution of all sports over years**

In [14]:
df_all_year = af.count_medals_n(athlete_iso, "Country", "ISO", "Year")
df_all_year = df_all_year.sort_values(by=["Year", "ISO"])
df_all_year.head()

Medal,Country,ISO,Year,Bronze,Gold,Silver,Total
33,Australia,AUS,1896,1,2,0,3
66,Austria,AUT,1896,2,2,1,5
1169,Switzerland,CHE,1896,0,1,2,3
452,Germany,DEU,1896,2,25,5,32
305,Denmark,DNK,1896,3,1,2,6


In [15]:
for medal in ["Total", "Gold", "Silver","Bronze"]:

    fig = px.choropleth(df_all_year, locations="ISO",
                        color=medal,
                        scope=None,
                        hover_name="Country",
                        animation_frame="Year",
                        title = f"Geographic map on {medal} medals over years", 
                        range_color=[0,df_all_year[medal].quantile(0.95)],
                        color_continuous_scale=px.colors.sequential.Plasma)
    
    
    fig["layout"].pop("updatemenus")
    fig.show()
    fig.write_html(f'Visualiseringar/Q2.plot_{medal}_over_year_on_map.html', auto_open=True)

**4.Medal distribution of specific sport over years**

In [16]:
athlete_iso = pd.read_csv("data/athlete_iso.csv").iloc[:, 1:]
df_sports_year = af.count_medals(athlete_iso, "Country", "ISO", "Sport","Year").reset_index()
df_sports_year = df_sports_year.sort_values(by=["Year", "ISO"])
df_sports_year.head()

Medal,Country,ISO,Sport,Year,Bronze,Gold,Silver,Total
3679,Australia,AUS,Athletics,1896,0,2,0,2
5558,Australia,AUS,Tennis,1896,1,0,0,1
3223,Austria,AUT,Cycling,1896,2,1,0,3
3775,Austria,AUT,Swimming,1896,0,1,1,2
3116,Switzerland,CHE,Gymnastics,1896,0,1,2,3


#TODO: choropleth cannot show in html

In [17]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
medal = "Total"
for sport in sports:
    df_sport_year = df_sports_year[df_sports_year["Sport"] == sport]
    fig = px.choropleth(df_sport_year, locations="ISO",
                        color=medal,
                        scope=None,
                        hover_name="Country",
                        animation_frame="Year",
                        title = f"Geographic map on {sport} {medal} medals over years", 
                        range_color=[0,df_sport_year[medal].quantile(0.95)],
                        color_continuous_scale=px.colors.sequential.Plasma)
    
    
    fig["layout"].pop("updatemenus")
    fig.show()
    fig.write_html(f'Visualiseringar/Q2.plot_{sport}_{medal}_over_year_on_map.html', auto_open=True)


**4.Medal Distribution over ages for only winners**

In [18]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
for sport in sports:
    df_sport = athlete_regions[athlete_regions["Sport"] == sport]

    # use sub_df module to get sub dataframe of choosen arg: "region"
    # select the countries which had medals in history and sort by total medals descending
    medals_sport= af.count_medals_n(df_sport, "Age")
    print(f"The ages win medals of {sport} are: {medals_sport.index.unique()}")
    print(f"The most common ages to win medals in Olympics are:")
    print(medals_sport.head(10))

    # plot:

    fig = px.histogram(medals_sport, x="Age", y=medals_sport.columns,
             title=f"Distribution of {sport} winners over ages")
    fig.update_layout(barmode='group')
    fig.show()
    fig.write_html(
            f"Visualiseringar/Q2.{sport}_age_distribution_winner.html", auto_open=True)

The ages win medals of Ice Hockey are: RangeIndex(start=0, stop=28, step=1)
The most common ages to win medals in Olympics are:
Medal   Age  Bronze  Gold  Silver  Total
0      15.0       2     0       0      2
1      16.0       2     0       4      6
2      17.0       2     0       1      3
3      18.0       7     6       3     16
4      19.0      15     9      15     39
5      20.0      33    20      20     73
6      21.0      26    28      31     85
7      22.0      45    41      46    132
8      23.0      30    52      59    141
9      24.0      46    49      50    145


The ages win medals of Tennis are: RangeIndex(start=0, stop=28, step=1)
The most common ages to win medals in Olympics are:
Medal   Age  Bronze  Gold  Silver  Total
0      16.0       0     1       0      1
1      17.0       1     0       2      3
2      18.0       1     3       3      7
3      19.0       3     2       3      8
4      20.0      11     7       4     22
5      21.0       6     6       2     14
6      22.0       8     5       6     19
7      23.0      11    10       7     28
8      24.0      11    12       2     25
9      25.0      10     6       7     23


The ages win medals of Swimming are: RangeIndex(start=0, stop=28, step=1)
The most common ages to win medals in Olympics are:
Medal   Age  Bronze  Gold  Silver  Total
0      12.0       1     0       0      1
1      13.0       2     1       4      7
2      14.0      10    13      13     36
3      15.0      17    39      33     89
4      16.0      52    56      49    157
5      17.0      69    93      74    236
6      18.0      95   105      93    293
7      19.0      90   115     104    309
8      20.0      99   108     105    312
9      21.0     129   151     113    393


The ages win medals of Football are: RangeIndex(start=0, stop=23, step=1)
The most common ages to win medals in Olympics are:
Medal   Age  Bronze  Gold  Silver  Total
0      15.0       1     0       1      2
1      16.0       0     1       3      4
2      17.0      11     2       6     19
3      18.0      16     5      11     32
4      19.0      20    22      18     60
5      20.0      28    31      33     92
6      21.0      58    35      54    147
7      22.0      67    72      84    223
8      23.0      68    66      59    193
9      24.0      52    44      48    144


**Age distribution among all athletes**

In [19]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
for sport in sports:
    df_sport = athlete_regions[athlete_regions["Sport"] == sport]
    df_athlete = df_sport.copy()
    df_athlete = df_athlete[df_athlete["Age"].notna()]
    athlete_counts = df_athlete["Age"].value_counts()
    fig = px.bar(athlete_counts, title=f"Age distribution of {medal} total athletes in {sport}")
    fig.update_layout(
        xaxis_title = "Age",
        yaxis_title = "Frequency",
        title_x = 0.5, 
        #showlegend = False
    )

    fig.show()
    fig.write_html(
            f"Visualiseringar/Q2.{sport}_age_distribution_all_athletes.html", auto_open=True)




**Medal distribution among gender**

In [20]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
for sport in sports:
    df_sport = athlete_regions[athlete_regions["Sport"] == sport]
    print(f"The dataframe has a shape: {df_sport.shape}")

    # use sub_df module to get sub dataframe of choosen arg: "region"
    # select the countries which had medals in history and sort by total medals descending
    medals_sport= af.count_medals(df_sport, "Sex")
    print(f"The gender of winners of medals in Olympics are:")
    print(medals_sport.head(10))

    # plot:

    fig = px.bar(medals_sport, x=medals_sport.index, y=medals_sport.columns,
             title=f"Distribution of {sport} winners over gender")
    fig.update_layout(barmode='group')
    fig.show()
    fig.write_html(
            f"Visualiseringar/Q2.{sport}_winners_gender_distribution.html", auto_open=True)

The dataframe has a shape: (5516, 18)
The gender of winners of medals in Olympics are:
Medal  Bronze  Gold  Silver  Total
Sex                               
M         409   407     414   1230
F          98   101     101    300


The dataframe has a shape: (2862, 18)
The gender of winners of medals in Olympics are:
Medal  Bronze  Gold  Silver  Total
Sex                               
M          76    62      62    200
F          52    44      44    140


The dataframe has a shape: (23195, 18)
The gender of winners of medals in Olympics are:
Medal  Bronze  Gold  Silver  Total
Sex                               
M         527   606     541   1674
F         429   493     452   1374


The dataframe has a shape: (6745, 18)
The gender of winners of medals in Olympics are:
Medal  Bronze  Gold  Silver  Total
Sex                               
M         441   414     414   1269
F         102   101      99    302


- The gender differences are quite large in Ice hockey and football games, compare to tennis and swimming.

**Gender distribution of all athletes in a specific sport**

In [21]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
for sport in sports:
    df_sport = athlete_regions[athlete_regions["Sport"] == sport]
    df_athlete = df_sport.copy()
    df_athlete = df_athlete[df_athlete["Sex"].notna()]
    athlete_counts = df_athlete["Sex"].value_counts()
    fig = px.bar(athlete_counts, title=f"Gender distribution of {medal} total athletes in {sport}")
    fig.update_layout(
        xaxis_title = "Gender",
        yaxis_title = "Frequency",
        title_x = 0.5, 
        #showlegend = False
    )

    fig.show()
    fig.write_html(
            f"Visualiseringar/Q2.{sport}_gender_distribution_all_athletes.html", auto_open=True)

**Medal distribution over height**
- show gender difference

In [22]:
sports = ["Ice Hockey", "Tennis", "Swimming", "Football" ]
for sport in sports:
    df_sport = athlete_regions[athlete_regions["Sport"] == sport]
    print(f"The dataframe has a shape: {df_sport.shape}")

    # use sub_df module to get sub dataframe of choosen arg: "region"
    # select the countries which had medals in history and sort by total medals descending
    medals_sport= af.count_medals_n(df_sport, "Sex", "Height")
    print(f"The common heigts of winners of medals in Olympics are:")
    print(medals_sport.head(10))
    
    medals_sport_f = medals_sport[medals_sport["Sex"]=="F"]

    fig = px.bar(medals_sport_f, x="Height", y=["Gold", "Silver", "Bronze", "Total"],
                title=f"Distribution of {sport} medals over height for female")
    fig.update_layout(barmode='group')
    fig.show()
    fig.write_html(
                f"Visualiseringar/Q2.{sport}_height_female.html", auto_open=True)

    medals_sport_m = medals_sport[medals_sport["Sex"]=="M"]
    fig = px.histogram(medals_sport_m, x="Height", y=["Gold", "Silver", "Bronze", "Total"],
                title=f"Distribution of {sport} medals over height for male")
    fig.update_layout(barmode='group')
    fig.show()
    fig.write_html(
                f"Visualiseringar/Q2.{sport}_height_male.html", auto_open=True)

The dataframe has a shape: (5516, 18)
The common heigts of winners of medals in Olympics are:
Medal Sex  Height  Bronze  Gold  Silver  Total
0       F   152.0       0     0       1      1
1       F   153.0       1     0       0      1
2       F   155.0       0     1       0      1
3       F   157.0       1     1       2      4
4       F   159.0       4     0       0      4
5       F   160.0       6     1       2      9
6       F   161.0       1     0       0      1
7       F   162.0       8     0       0      8
8       F   163.0       8    13       7     28
9       F   164.0       3     1       0      4


The dataframe has a shape: (2862, 18)
The common heigts of winners of medals in Olympics are:
Medal Sex  Height  Bronze  Gold  Silver  Total
0       F   162.0       1     1       0      2
1       F   163.0       1     0       1      2
2       F   164.0       3     0       2      5
3       F   165.0       1     0       0      1
4       F   166.0       1     0       0      1
5       F   167.0       0     1       0      1
6       F   168.0       0     1       0      1
7       F   169.0       3     0       4      7
8       F   170.0       2     3       4      9
9       F   171.0       2     0       0      2


The dataframe has a shape: (23195, 18)
The common heigts of winners of medals in Olympics are:
Medal Sex  Height  Bronze  Gold  Silver  Total
0       F   142.0       1     0       0      1
1       F   150.0       1     0       0      1
2       F   152.0       0     0       1      1
3       F   155.0       0     0       1      1
4       F   157.0       1     1       1      3
5       F   158.0       2     0       0      2
6       F   159.0       1     2       0      3
7       F   160.0       6     9       7     22
8       F   161.0       3     1       2      6
9       F   162.0       3     4       7     14


The dataframe has a shape: (6745, 18)
The common heigts of winners of medals in Olympics are:
Medal Sex  Height  Bronze  Gold  Silver  Total
0       F   153.0       2     1       1      4
1       F   154.0       0     0       1      1
2       F   155.0       0     1       3      4
3       F   156.0       1     0       1      2
4       F   157.0       0     0       4      4
5       F   158.0       0     1       0      1
6       F   159.0       0     0       1      1
7       F   160.0       5     2       2      9
8       F   161.0       1     0       3      4
9       F   162.0       0     0      10     10
